In [1]:
from utils import Activities, Users

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.options.plotting.backend = "plotly"

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from utils import join_by_fuzzy

import info_utilities

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from pprint import pprint

import pydot
from IPython.display import Image

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows    = None

In [3]:
data_folder = "../data/"

In [4]:
users = Users(f"{data_folder}users.csv")
users.fillna()

students = users.df.query("student == 1")
students = students.drop_duplicates('user_name',keep='first')

# TODO: merge redoundant users
supervisors = users.df.query("supervisor == 1")
teachers  = users.df.query("teacher == 1")

In [6]:
first = users.df[users.df.duplicated(['user_email'],keep='first')].sort_values(by="user_email")['us_user']
last  = users.df[users.df.duplicated(['user_email'],keep='last')].sort_values(by="user_email")['us_user']
correct_map = dict(zip(first,last))

In [7]:
to_sum = ['n_activities', 'n_activities_school_year_1',
       'n_activities_school_year_2', 'n_activities_school_year_3', 'n_recipes',
       'n_experiences', 'n_reflections', 'n_recipe_reflections',
       'n_experience_reflections', 'n_in_curriculum',
       'n_recipes_in_curriculum', 'n_experiences_in_curriculum',
       'n_in_curriculum_semester1', 'n_in_curriculum_semester2',
       'n_in_curriculum_semester3', 'n_in_curriculum_semester4',
       'n_in_curriculum_semester5', 'n_feedback_requests',
       'n_received_feedback_responses', 'n_received_feedback_requests',
       'n_feedback_responses', 'n_files', 'n_folders']
to_avg = ['avg_activity_evaluations',
       'avg_reflection_length', 'avg_specific_evaluations',
       'avg_supervisor_evaluation']
for new_id, old_id in correct_map.items():
    m = users.df['us_user'] == old_id
    n = users.df['us_user'] == new_id
    users.df.loc[m,to_sum] = users.df.loc[m,to_sum].values + users.df.loc[n,to_sum].values
    users.df.loc[m,to_avg] = (users.df.loc[m,to_avg].values + users.df.loc[n,to_avg].values)/2
    
to_drop = correct_map.keys()
users.df = users.df.drop(users.df[users.df['us_user'].isin(to_drop)].index)